In [ ]:
import os
import pandas as pd
import numpy as np
import fitsio
import cupy as cp
from pathlib import Path, PurePath

In [ ]:
def get_dist_mat(coords):
    coords_x = coords % 4096
    coords_y = coords // 4096
    coords_xb = coords_x[:, np.newaxis]
    coords_yb = coords_y[:, np.newaxis]
    dx_broadc = coords_x - coords_xb
    dy_broadc = coords_y - coords_yb
    dist_matrix = dx_broadc**2 + dy_broadc**2
    return dist_matrix

def get_dist_mat_gpu(coords):
    coords_x = coords % 4096
    coords_y = coords // 4096
    coords_xb = coords_x[:, cp.newaxis]
    coords_yb = coords_y[:, cp.newaxis]
#     dx_broadc = coords_x - coords_xb
#     dy_broadc = coords_y - coords_yb
    dist_matrix = (coords_x - coords_xb)**2 + (coords_y - coords_yb)**2
    cp.cuda.runtime.deviceSynchronize()
    return dist_matrix


@cp.fuse()
def dist_mat(x1, x2, y1, y2):
    return (x1 - x2)*(x1 - x2) + (y1-y2)*(y1-y2)


def get_dist_mat_gpu_fuse(coords):
    coords_x = coords % 4096
    coords_y = coords // 4096
    coords_xb = coords_x[:, cp.newaxis]
    coords_yb = coords_y[:, cp.newaxis]
    dist_matrix = dist_mat(coords_x, coords_xb, coords_y, coords_yb)
    cp.cuda.runtime.deviceSynchronize()
    return dist_matrix



def get_rows_list(array, w1_idx, w2_idx):
    np_mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
    df_idx = np.nonzero(np_mask)[0]
    coords = array[np_mask, 0]
    dist_matrix = get_dist_mat(coords)
    select = dist_matrix < 2 
    select2 = np.triu(select, k=1)
    r,c = np.nonzero(select2)
    idx1, idx2 = df_idx[r], df_idx[c]
    return idx1, idx2

def get_rows_list_gpu(array, w1_idx, w2_idx):
    mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
    df_idx = cp.nonzero(mask)[0]
    coords = array[df_idx, 0]
    dist_matrix = get_dist_mat_gpu(coords)
    select = dist_matrix < 2 
    select2 = cp.triu(select, k=1)
    r,c = cp.nonzero(select2)
    idx1, idx2 = df_idx[r], df_idx[c]
    return idx1, idx2


def get_2coincidentals(array, w1_idx, w2_idx):
    idx1, idx2 = get_rows_list(array, w1_idx, w2_idx)
    records = [df0.loc[[i1, i2]][['coords', 'int1', 'int2', 'wref']] for i1, i2 in zip(idx1, idx2)]
    df_records = pd.concat(records, keys=list(range(len(records))))
    return df_records


def get_2clusters(array, w1_idx, w2_idx):
    idx1, idx2 = get_rows_list(array, w1_idx, w2_idx)
    # To remove conjugates
#     keep_mask = arr0[idx1, 3] != arr0[idx2, 3] 
#     recordsf = np.concatenate((arr0[idx1[keep_mask], 0:4], arr0[idx2[keep_mask], 0:4]), axis=1)
    
    records = np.concatenate((array[idx1, 0:4], array[idx2, 0:4]), axis=1)
    df_records = pd.DataFrame(records, columns=['coords1', 'int1_before', 'int1_after', 'wref1', 'coords2', 'int2_before', 'int2_after', 'wref2'])

    return df_records


def get_2clusters_gpu(gpu_array, w1_idx, w2_idx):
    gidx1, gidx2 = get_rows_list_gpu(gpu_array, w1_idx, w2_idx)
    grecords = cp.concatenate([gpu_array[gidx1, 0:4], gpu_array[gidx2, 0:4]], axis=1)
    #grecords2 = cp.asfortranarray(grecords)
    #cudf_records = cudf.DataFrame.from_gpu_matrix(grecords2, columns=['coords1', 'int1_before', 'int1_after', 'wref1', 'coords2', 'int2_before', 'int2_after', 'wref2'])
    return records

In [ ]:
# data_dir = os.environ['SPIKESDATA']
# spikes_db = pd.read_parquet(os.path.join(data_dir, 'spikes_df_2010.parquet'), engine='pyarrow')
# spikes_db2 = spikes_db.set_index(['GroupNumber', 'Time'])

### Get the filepaths (typically 7) for a given group

In [ ]:
parquet_dir = os.path.expanduser('~/raphael/data/AIA_Spikes/parquet_dataframes')
df0 = pd.read_parquet(PurePath(parquet_dir, '2010/05/df_coincidentals_2010_05_13.parquet'))
df0.set_index('GroupNumber', inplace=True)
df0.loc[0].head()

### Design method to extract unique coincidental events, lifting any ambiguity (conjugates, redundancies, ...)

In [ ]:
w1_idx = 4
w2_idx = 5
df_ = df0.loc[range(50)] # This is very slow on HP Z8. Seems faster on Dell laptop. Need to test again
print(len(df_))
df_.iloc[-10:-5]

In [ ]:
array = df_.values
np_mask = (array[:, w1_idx] == 1) & (array[:, w2_idx] == 1)
df_idx = np.nonzero(np_mask)[0]
coords = array[np_mask, 0]

In [ ]:
dist_matrix = get_dist_mat(coords)

In [ ]:
print(dist_matrix.shape)
dist_matrix.nbytes/(1024*1e3)

In [ ]:
%timeit dist_matrix = get_dist_mat(coords)

## Testing GPU versions

In [ ]:
gcoords = cp.asarray(coords)

In [ ]:
gdist_matrix = get_dist_mat_gpu(gcoords)
np.array_equal(d1, dist_matrix)

In [ ]:
%timeit gdist_matrix = get_dist_mat_gpu(gcoords)

In [ ]:
gdist_matrix2 = get_dist_mat_gpu_fuse(gcoords)

In [ ]:
%timeit gdist_matrix2 = get_dist_mat_gpu_fuse(gcoords)

In [ ]:
d1 = gdist_matrix.get()
d2 = gdist_matrix2.get()

In [ ]:
np.array_equal(d1, d2)